# 1
Попробуйте улучшить точность распознавания образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложите анализ с описанием того, что улучшает работу нейронной сети, а что ухудшает  
# 2
Опишите в анализе, какие изменения необходимо было бы внести в получившуюся нейронную сеть, если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET

# 2
С MNIST будет неплохо работа и та архитектура что и у cifar Предположительно можно будет упростить архитиктуру и уменьшить количество слоев. input 28, 28 a output одинаков. В случае использования уже готовых архитектур необходимо добавить 1 простраснство в данные для признаков.  
C CIFAR100  потребуется больше слоев свертки так же поменяется форма output будет 100  
С IMAGENET скорей всего потребуется еще больше слоев свертки так же меняется форма ввода  469x387 или исользуя препроцесинг и понижение размерности
В модели хорошой идее будет добавить бач нормализацию что бы лос не становлся неопределенным

In [ ]:
#2
С MNIST будет неплохо работа и та архитектура что и у cifar Предположительно можно будет упростить архитиктуру и уменьшить количество слоев. input 28, 28 a output одинаков. В случае использования уже готовых архитектур необходимо добавить 1 простраснство в данные для признаков.  
C CIFAR100  потребуется больше слоев свертки так же поменяется форма инпута в оутпуте будет 100  
С IMAGENET скорей всего потребуется еще больше слоев свертки так же меняется форма ввода  469x387 или исользуя препроцесинг и понижение размерности
В модели хорошой идее будет добавить бач нормализацию что бы лос не становлся неопределенным

# Вариант по умолчанию

In [20]:
from __future__ import print_function
import keras

from keras.applications import ResNet50
from keras.applications import ResNet152
from keras.applications import VGG19

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
import os

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        vertical_flip=False)

# Установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 1

train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)
test_generator = datagen.flow(x_test, y_test, batch_size=batch_size)

# Создание предварительно обученной модели ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Создание модели для Featurization
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

# Компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Обучение модели
model.fit(train_generator,
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=len(x_test) // batch_size)

# Оценка производительности модели на тестовых данных
score = model.evaluate(test_generator, steps=len(x_test) // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

312/312 [==============================] - 8s 24ms/step - loss: 168.6040 - accuracy: 0.0000e+00
Test loss: 168.60403442382812
Test accuracy: 0.0


Очевидно что нехватает эпох обучения а так же использовать ImageDataGenerator лучше после того как определимся с базовой архитекторуй. Ниже посмотрим 3 архитиктуры на 3 эпохах (орграничение связаное с мощностями)

In [3]:
from __future__ import print_function
import keras

from keras.applications import ResNet50
from keras.applications import ResNet152
from keras.applications import VGG19

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
import os
from keras import models, layers
import keras
from keras.utils import to_categorical


In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# Установка параметров нейросети
batch_size = 14
num_classes = 10
epochs = 3
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)


# Создание предварительно обученной модели ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Создание модели для Featurization
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

# Компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Обучение модели
model.fit(x=x_train, y=y_train, batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

# Оценка производительности модели на тестовых данных
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/3
3572/3572 [==============================] - 174s 39ms/step - loss: 2.4125 - accuracy: 0.2170 - val_loss: 2.3145 - val_accuracy: 0.2336
Epoch 2/3
3572/3572 [==============================] - 138s 39ms/step - loss: 2.1624 - accuracy: 0.2548 - val_loss: 1.6757 - val_accuracy: 0.3674
Epoch 3/3
313/313 [==============================] - 4s 11ms/step - loss: 1.4503 - accuracy: 0.4738
Test loss: 1.450250506401062
Test accuracy: 0.47380000352859497


In [12]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# Установка параметров нейросети
batch_size = 14
num_classes = 10
epochs = 3
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)


# Создание предварительно обученной модели ResNet50
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Создание модели для Featurization
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

# Компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Обучение модели
model.fit(x=x_train, y=y_train, batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

# Оценка производительности модели на тестовых данных
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

234698864/234698864 [==============================] - 2s 0us/step
Epoch 1/3
3572/3572 [==============================] - 515s 114ms/step - loss: 2.5767 - accuracy: 0.1291 - val_loss: 2.3741 - val_accuracy: 0.1001
Epoch 2/3
3572/3572 [==============================] - 401s 112ms/step - loss: 2.4313 - accuracy: 0.1405 - val_loss: 2.1011 - val_accuracy: 0.1629
Epoch 3/3
313/313 [==============================] - 8s 25ms/step - loss: 2.0760 - accuracy: 0.2247
Test loss: 2.076000213623047
Test accuracy: 0.22470000386238098


In [14]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# Установка параметров нейросети
batch_size = 14
num_classes = 10
epochs = 3
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

# Создание предварительно обученной модели ResNet50
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Создание модели для Featurization
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

# Компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Обучение модели
model.fit(x=x_train, y=y_train, batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

# Оценка производительности модели на тестовых данных
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

80134624/80134624 [==============================] - 1s 0us/step
Epoch 1/3
3572/3572 [==============================] - 113s 30ms/step - loss: 2.3223 - accuracy: 0.1007 - val_loss: 2.3038 - val_accuracy: 0.1000
Epoch 2/3
3572/3572 [==============================] - 105s 29ms/step - loss: 2.3036 - accuracy: 0.0985 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/3
313/313 [==============================] - 4s 11ms/step - loss: 2.3026 - accuracy: 0.1000
Test loss: 2.3026206493377686
Test accuracy: 0.10000000149011612


In [16]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 global_average_pooling2d_5  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 10)                5130      
                                                                 
Total params: 20029514 (76.41 MB)
Trainable params: 20029514 (76.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### ResNet50 быстрее спускается к оптимуму в отличее от других архитектур

# Более компактный вариант записи 

In [ ]:
base_model1 = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model2 = ResNet152(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model3 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
b_model_list = [base_model1, base_model2, base_model3]
names = ['VGG19','ResNet152', 'ResNet50']
for base_model, name in (models, name):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    # Установка параметров нейросети
    batch_size = 14
    num_classes = 10
    epochs = 3
    y_test = to_categorical(y_test)
    y_train = to_categorical(y_train)

    # Создание модели для Featurization
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(num_classes, activation='softmax'))

    # Компиляция модели
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

    # Обучение модели
    model.fit(x=x_train, y=y_train, batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

    # Оценка производительности модели на тестовых данных
    score = model.evaluate(x_test, y_test)
    print('name')
    model.summary()
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
        

# Своя архитектура

In [19]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout
# Set the neural network parameters
batch_size = 20
num_classes = 10
epochs = 3
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_model.h5'

# Load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'training examples')
print(x_test.shape[0], 'testing examples')


# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

#model.add(Conv2D(256, (3, 3), activation="relu", padding="same"))
#model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
#model.add(Dense(128, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.summary()
# Компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Обучение модели
model.fit(x=x_train, y=y_train, batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

# Оценка производительности модели на тестовых данных
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (50000, 32, 32, 3)
50000 training examples
10000 testing examples
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                       

2023-11-21 09:10:27.512137: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_8/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2500/2500 [==============================] - 21s 7ms/step - loss: 1.3276 - accuracy: 0.5206 - val_loss: 1.1444 - val_accuracy: 0.6000
Epoch 2/3
2500/2500 [==============================] - 16s 7ms/step - loss: 0.9002 - accuracy: 0.6841 - val_loss: 0.8839 - val_accuracy: 0.6954
Epoch 3/3
313/313 [==============================] - 1s 4ms/step - loss: 0.7625 - accuracy: 0.7370
Test loss: 0.7625251412391663
Test accuracy: 0.7369999885559082


### При большем количестве эпох удалось бы добится большей точности.